# Global Budget Spending Analysis
----
<i> Project 1 Team 5 </i><br/> 
Brett Fuller, Diego Jones, Mav Sanchez

In [1]:
#Import necessary package dependencies
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from pprint import pprint
from scipy.stats import linregress

#Import API Key
from config import api_key

#Define directories
source_data_dir = "source_data"
staging_data_dir = "staging_data"
target_data_dir = "target_data"
target_images_dir = "target_images"

### Data extraction
----
#####            API Calls

In [52]:
budget_breakdown = "source_data/budget_breakdown.csv"

budgetbreakdown_df = pd.read_csv(budget_breakdown)
clean_budgetbreakdown = budgetbreakdown_df[['Unnamed: 0', 'Expenditure on general public services',
       'Expenditure on defense', 'Expenditure on public order & safety',
       'Expenditure on economic affairs',
       'Expenditure on environment protection',
       'Expenditure on housing & community amenities', 'Expenditure on health',
       'Expenditure on recreation, culture, & religion',
       'Expenditure on education', 'Expenditure on social protection']]
clean_budgetbreakdown = clean_budgetbreakdown.dropna()

clean_budgetbreakdown = clean_budgetbreakdown.rename(columns = {'Unnamed: 0': 'Country'})





In [57]:
country_rank = "source_data/top_73.csv"
country_ranking = pd.read_csv(country_rank, header=None
                             )
country_ranking = country_ranking.rename(columns = {0: 'Country'})

combined_countrydf = clean_budgetbreakdown.merge(country_ranking, on='Country')
combined_countrydf.columns = ['Country',
               'Expenditure on general public services',
                               'Expenditure on defense',
                 'Expenditure on public order & safety',
                      'Expenditure on economic affairs',
                'Expenditure on environment protection',
         'Expenditure on housing & community amenities',
                                'Expenditure on health',
       'Expenditure on recreation, culture, & religion',
                             'Expenditure on education',
                     'Expenditure on social protection',
                                                      'Overall Ranking',
                                                      'Entrepreneurship Ranking',
                                                      'Adventure Ranking',
                                                      'Citizenship Ranking',
                                                      'Cultural Influence Ranking',
                                                      'Heritage Ranking',
                                                      'Movers Ranking',
                                                      'Open For Business Ranking',
                                                      'Power Ranking',
                                                     'Quality of Life Ranking']

combined_countrydf.head()


,Country,Expenditure on general public services,Expenditure on defense,Expenditure on public order & safety,Expenditure on economic affairs,Expenditure on environment protection,Expenditure on housing & community amenities,Expenditure on health,"Expenditure on recreation, culture, & religion",Expenditure on education,...,Overall Ranking,Entrepreneurship Ranking,Adventure Ranking,Citizenship Ranking,Cultural Influence Ranking,Heritage Ranking,Movers Ranking,Open For Business Ranking,Power Ranking,Quality of Life Ranking
0,Austria,6.4,0.6,1.3,5.7,0.4,0.3,8.2,1.2,4.9,...,18,17,22,12,25,20,59,15,27,11
1,Estonia,4.0,2.4,1.9,4.2,0.6,0.3,5.2,2.0,5.7,...,55,40,60,26,61,62,73,26,72,45
2,Finland,8.0,1.3,1.2,4.5,0.2,0.3,7.2,1.4,6.1,...,14,16,27,6,26,43,32,9,35,9
3,France,6.2,1.8,1.6,5.5,0.9,1.1,8.1,1.4,5.5,...,12,15,12,13,2,4,44,29,6,16
4,Germany,5.7,0.9,1.6,3.3,0.6,0.4,7.2,1.0,4.1,...,4,1,50,10,15,19,41,17,4,10


In [65]:
population_breakdown = "source_data/Countries.csv"

populationbreakdown_df = pd.read_csv(population_breakdown)
populationbreakdown_df.columns


doublecombined_countrydf = combined_countrydf.merge(populationbreakdown_df, on='Country')
doublecombined_countrydf.head()

,Country,Expenditure on general public services,Expenditure on defense,Expenditure on public order & safety,Expenditure on economic affairs,Expenditure on environment protection,Expenditure on housing & community amenities,Expenditure on health,"Expenditure on recreation, culture, & religion",Expenditure on education,...,Movers Ranking,Open For Business Ranking,Power Ranking,Quality of Life Ranking,GDPPC,Literacy,InfantMortality,Agriculture,Population,NetMigration
0,Austria,6.4,0.6,1.3,5.7,0.4,0.3,8.2,1.2,4.9,...,59,15,27,11,43439,0.980,3.4,0.014,8611000,5.2
1,Estonia,4.0,2.4,1.9,4.2,0.6,0.3,5.2,2.0,5.7,...,73,26,72,45,17295,0.998,3.8,0.034,1312000,-3.2
2,Finland,8.0,1.3,1.2,4.5,0.2,0.3,7.2,1.4,6.1,...,32,9,35,9,41921,1.000,2.5,0.028,5482000,3.0
3,France,6.2,1.8,1.6,5.5,0.9,1.1,8.1,1.4,5.5,...,44,29,6,16,36248,0.990,3.3,0.017,66810000,1.1
4,Germany,5.7,0.9,1.6,3.3,0.6,0.4,7.2,1.0,4.1,...,41,17,4,10,41219,0.990,3.4,0.007,81410000,1.5


In [ ]:
# Define base URLs for API
base_url_fromwherever ="put main endpoint here"

In [ ]:
# Get  data
list_whatever = []
save_to_whatever = []

for a in list_whatever:
    
    query_url = base_url_fromwherever + 'appid=' + api_key + '&param=' + a
    response = requests.get(query_url).json() 
    try:
        list_whatever.append(response) #customize depending on the JSON structure
    except:
        pass

# Save api data to data frames
staging_df = pd.DataFrame(list_whatever)

In [ ]:
# Save df to csv so we don't have to perform API calls again
staging_df.to_csv(f"{staging_data_dir}/staging_df.csv")

In [4]:
#####            CSV Load

In [ ]:
# Load raw CSV files to data frame
raw_csv_data = pd.read_csv(f"{source_data_dir}/descriptive_file_name.csv")

### Data cleansing
----

In [ ]:
# Options
# Delete nulls
# FillNA with zero, etc
# Reformat data (date format, name format)
# Save only the columns you need

### Data analysis
----

#### Question 1: (Sample) What our developed countries' most prioritized spending?
Answer here

In [9]:
# Perform statistical analysis
# Put applicable plot depending on your question